# 🎨 BNNR Augmentations — Visual Guide

**Understand every augmentation that BNNR offers — and why ICD/AICD are game-changers.**

BNNR ships 8 built-in augmentations — **7 unique, hand-crafted transforms** (ChurchNoise, DifPresets, Drust, LuxferGlass, ProCAM, Smugs, TeaStains) that you won't find in torchvision, Albumentations, or Kornia, plus **BasicAugmentation** which bundles standard transforms (flips, rotations, crops) for convenience. On top of that, **ICD** (Intelligence Coarse Dropout) and **AICD** (Anti-Intelligence Coarse Dropout) use XAI saliency maps to intelligently decide *which parts of each image* to augment — something no other augmentation library offers.

This notebook:
1. Shows you every built-in augmentation **visually** on real images
2. Explains **what each one does** and **when to use it**
3. Deep-dives into **ICD & AICD** — BNNR's flagship innovation
4. Shows how to mix BNNR augmentations with torchvision/Albumentations/Kornia
5. Demonstrates how BNNR **automatically selects** the best augmentations for your task

⏱ **Runtime:** ~5 min (visualization only, no training).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mateuszwalo/BNNR/blob/main/examples/bnnr_augmentations_guide.ipynb)

## 1. Installation

In [ ]:
%pip install -q "bnnr[dashboard]" matplotlib

## 2. Imports & Helper

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from bnnr import (
    BasicAugmentation,
    ChurchNoise,
    DifPresets,
    Drust,
    LuxferGlass,
    ProCAM,
    Smugs,
    TeaStains,
    TorchvisionAugmentation,
    AugmentationRegistry,
    auto_select_augmentations,
)

from bnnr.icd import ICD, AICD
from bnnr.xai_cache import XAICache


def show_augmentation(original_np, aug, title, n_variants=4):
    """Show original + N augmented variants side by side."""
    fig, axes = plt.subplots(1, n_variants + 1, figsize=(3 * (n_variants + 1), 3))
    axes[0].imshow(original_np)
    axes[0].set_title("Original", fontsize=10)
    axes[0].axis("off")
    for i in range(n_variants):
        augmented = aug.apply(original_np.copy())
        axes[i + 1].imshow(augmented)
        axes[i + 1].set_title(f"Variant {i+1}", fontsize=10)
        axes[i + 1].axis("off")
    fig.suptitle(title, fontsize=13, fontweight="bold")
    plt.tight_layout()
    plt.show()


print("Ready!")

## 3. Load Sample Images

We use **STL-10** — 96×96 colour images of real objects. The higher resolution makes augmentation effects clearly visible.

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
dataset = datasets.STL10("data", split="train", download=True, transform=transform)

SAMPLE_INDICES = [0, 100, 500, 1200]
STL10_CLASSES = ["airplane", "bird", "car", "cat", "deer",
                 "dog", "horse", "monkey", "ship", "truck"]

samples = []
for idx in SAMPLE_INDICES:
    img_tensor, label = dataset[idx]
    img_np = (img_tensor.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
    samples.append((img_np, label))

fig, axes = plt.subplots(1, 4, figsize=(12, 3))
for i, (img, lbl) in enumerate(samples):
    axes[i].imshow(img)
    axes[i].set_title(STL10_CLASSES[lbl], fontsize=11)
    axes[i].axis("off")
fig.suptitle("Sample Images (STL-10)", fontsize=13, fontweight="bold")
plt.tight_layout()
plt.show()

---

## 4. The 8 Built-in BNNR Augmentations

7 of these are **unique to BNNR** — you won't find them in torchvision, Albumentations, or Kornia. Each simulates a different type of real-world image degradation or artistic effect. The 8th, **BasicAugmentation**, bundles standard CV transforms (flips, rotations, crops) for convenience.

### Why are they special?
- They're designed for **regularization** — making models robust to visual noise without destroying semantic content
- They generate **diverse, non-trivial** perturbations (not just flips and crops)
- They work on **any resolution** and **any number of channels** (after conversion to RGB)
- BNNR automatically tests each one and **keeps only those that improve your metric**

### 4.1 ChurchNoise

Simulates **optical noise patterns** inspired by stained-glass diffraction. Adds structured colour noise that varies across the image. Excellent for training models to ignore lighting artifacts and colour shifts.

**Best for:** Models that overfit to specific colour distributions.

In [ ]:
aug = ChurchNoise(probability=1.0, random_state=None)
show_augmentation(samples[0][0], aug, "ChurchNoise — Structured Colour Noise")

### 4.2 BasicAugmentation

A **composite** of fundamental transforms: random brightness, contrast, saturation shifts, plus mild geometric jitter. Think of it as a balanced "all-rounder" augmentation.

**Best for:** General-purpose regularization when you're not sure what augmentation to start with.

In [ ]:
aug = BasicAugmentation(probability=1.0, random_state=None)
show_augmentation(samples[1][0], aug, "BasicAugmentation — All-Round Composite")

### 4.3 DifPresets

Applies **differentiable preset-style** colour grading — similar to Instagram filters but parameterized for ML. Changes colour temperature, tone curves, and saturation in a photorealistic way.

**Best for:** Models processing photos from diverse cameras/lighting conditions.

In [ ]:
aug = DifPresets(probability=1.0, random_state=None)
show_augmentation(samples[2][0], aug, "DifPresets — Photorealistic Colour Grading")

### 4.4 Drust

Simulates **dust and scratch patterns** on camera lenses or old photographs. Adds thin lines and speckle patterns that partially occlude the image.

**Best for:** Robustness to physical sensor artifacts (scratches, dust, water droplets).

In [ ]:
aug = Drust(probability=1.0, random_state=None)
show_augmentation(samples[3][0], aug, "Drust — Dust & Scratch Simulation")

### 4.5 LuxferGlass

Simulates viewing through **frosted or textured glass** — creates local pixel displacement and blending. The image remains recognizable but gains a subtle diffusion effect.

**Best for:** Training invariance to partial occlusion and texture overlay.

In [ ]:
aug = LuxferGlass(probability=1.0, random_state=None)
show_augmentation(samples[0][0], aug, "LuxferGlass — Frosted Glass Effect")

### 4.6 ProCAM

Simulates **professional camera effects**: lens flare, vignetting, chromatic aberration, and exposure variation. Produces photorealistic degradation.

**Best for:** Outdoor/natural image models where lighting conditions vary wildly.

In [ ]:
aug = ProCAM(probability=1.0, random_state=None)
show_augmentation(samples[1][0], aug, "ProCAM — Camera Effect Simulation")

### 4.7 Smugs

Applies **smudge/smear-like** effects — local blurring and colour bleeding. Simulates finger smudges on a camera lens or motion blur artifacts.

**Best for:** Robustness to motion blur and physical lens contamination.

In [ ]:
aug = Smugs(probability=1.0, random_state=None)
show_augmentation(samples[2][0], aug, "Smugs — Smudge/Smear Effect")

### 4.8 TeaStains

Simulates **circular stain patterns** overlaid on the image — think coffee ring stains on a document. Adds semi-transparent circular artifacts of varying size and colour.

**Best for:** Document/medical imaging where physical stains are a real issue.

In [ ]:
aug = TeaStains(probability=1.0, random_state=None)
show_augmentation(samples[3][0], aug, "TeaStains — Circular Stain Overlay")

### All 8 on One Image — Side by Side

In [ ]:
all_augs = [
    ("ChurchNoise", ChurchNoise(probability=1.0, intensity=0.5, random_state=42)),
    ("BasicAug", BasicAugmentation(probability=1.0, intensity=0.5, random_state=42)),
    ("DifPresets", DifPresets(probability=1.0, intensity=0.7, random_state=42)),
    ("Drust", Drust(probability=1.0, intensity=0.5, random_state=42)),
    ("LuxferGlass", LuxferGlass(probability=1.0, intensity=0.5, random_state=42)),
    ("ProCAM", ProCAM(probability=1.0, random_state=42)),
    ("Smugs", Smugs(probability=1.0, intensity=1.5, random_state=42)),
    ("TeaStains", TeaStains(probability=1.0, intensity=0.5, random_state=42)),
]

img = samples[0][0]
fig, axes = plt.subplots(2, 5, figsize=(18, 7))
axes[0, 0].imshow(img)
axes[0, 0].set_title("Original", fontsize=11, fontweight="bold")
axes[0, 0].axis("off")
axes[1, 0].axis("off")

for i, (name, aug) in enumerate(all_augs):
    row, col = divmod(i, 4)
    ax = axes[row, col + 1]
    ax.imshow(aug.apply(img.copy()))
    ax.set_title(name, fontsize=10)
    ax.axis("off")

fig.suptitle("All 8 BNNR Built-in Augmentations — Unique Effects Not Available Elsewhere",
             fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

---

## 5. 🌟 ICD & AICD — BNNR's Flagship Innovation

This is what makes BNNR fundamentally different from every other augmentation library.

### The Problem
Traditional augmentations (flip, rotate, colour jitter) are **random** — they don't know which parts of the image the model actually looks at. A random crop might remove the exact feature the model needs to learn, or leave an irrelevant background area untouched.

### The Solution: Saliency-Guided Augmentation
ICD and AICD use **XAI saliency maps** (generated by OptiCAM/GradCAM) to identify which image regions the model considers important. Then:

| Augmentation | Masks | Effect | Why It Helps |
|:---|:---|:---|:---|
| **ICD** (Intelligence Coarse Dropout) | **Important regions** (where model looks) | Forces model to learn from context | Prevents over-reliance on single features |
| **AICD** (Anti-Intelligence Coarse Dropout) | **Unimportant regions** (background) | Reduces background noise | Focuses learning on discriminative features |

### How It Works (Step by Step)

```
Image → XAI Saliency Map → Tile Grid → Threshold → Mask → Fill

1. Generate saliency map (OptiCAM) — which pixels matter to the model?
2. Divide image into tiles (e.g. 8×8 grid)
3. Compute mean saliency per tile
4. ICD:  mask tiles where saliency > threshold (important areas)
   AICD: mask tiles where saliency < threshold (unimportant areas)
5. Fill masked tiles with blurred original, noise, or local mean
```

The **fill strategy** is crucial — instead of black rectangles (which create out-of-distribution artifacts), BNNR uses **gaussian blur** of the original pixels, preserving low-frequency structure.

### 5.1 Set Up XAI for ICD

ICD needs a model and its saliency maps. Let's create a simple model and precompute the XAI cache.

In [ ]:
from torch.utils.data import Dataset, Subset


class IndexedDataset(Dataset):
    """Wraps (img, label) -> (img, label, index) for XAI cache."""
    def __init__(self, ds):
        self.ds = ds
    def __len__(self):
        return len(self.ds)
    def __getitem__(self, idx):
        img, lbl = self.ds[idx]
        return img, lbl, idx


class DemoNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128), nn.ReLU(True),
            nn.AdaptiveAvgPool2d((1, 1)),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(), nn.Linear(128, 10),
        )

    def forward(self, x):
        return self.classifier(self.features(x))

    @property
    def target_layer(self):
        return self.features[8]


model = DemoNet()
target_layers = [model.target_layer]

small_ds = Subset(dataset, range(200))
small_loader = DataLoader(IndexedDataset(small_ds), batch_size=32, shuffle=False)

print("Precomputing XAI saliency maps for ICD/AICD...")
xai_cache = XAICache(cache_dir="xai_cache_demo")
written = xai_cache.precompute_cache(
    model=model,
    train_loader=small_loader,
    target_layers=target_layers,
    n_samples=200,
    method="opticam",
)
print(f"Cached {written} saliency maps")

### 5.2 Visualize ICD — Masking Important Regions

ICD identifies where the model focuses attention (saliency hotspots) and masks those regions. The model is forced to learn from **context and secondary features** instead of relying on a single dominant cue.

In [ ]:
from bnnr.xai import generate_saliency_maps

fig, axes = plt.subplots(3, 4, figsize=(14, 10))

icd = ICD(
    model=model, target_layers=target_layers,
    threshold_percentile=60.0, explainer="opticam",
    use_cuda=False, cache=xai_cache,
    tile_size=8, fill_strategy="gaussian_blur",
    probability=1.0,
)

for col, idx in enumerate(SAMPLE_INDICES[:4]):
    img_tensor, label = dataset[idx]
    img_np = (img_tensor.permute(1, 2, 0).numpy() * 255).astype(np.uint8)

    with torch.no_grad():
        maps = generate_saliency_maps(
            model, img_tensor.unsqueeze(0),
            torch.tensor([label]), target_layers, method="opticam",
        )
    saliency = maps[0].numpy()

    imgs_batch = np.expand_dims(img_np, 0)
    labels_batch = np.array([label])
    indices_batch = np.array([idx])
    icd_result = icd.apply_batch_with_labels(imgs_batch.copy(), labels_batch, indices_batch)

    axes[0, col].imshow(img_np)
    axes[0, col].set_title(STL10_CLASSES[label], fontsize=10)
    axes[0, col].axis("off")

    axes[1, col].imshow(saliency, cmap="hot")
    axes[1, col].set_title("Model attention", fontsize=10)
    axes[1, col].axis("off")

    axes[2, col].imshow(icd_result[0])
    axes[2, col].set_title("After ICD", fontsize=10)
    axes[2, col].axis("off")

axes[0, 0].set_ylabel("Original", fontsize=11, fontweight="bold")
axes[1, 0].set_ylabel("Saliency\n(model attention)", fontsize=11, fontweight="bold")
axes[2, 0].set_ylabel("ICD Result\n(important masked)", fontsize=11, fontweight="bold")

fig.suptitle("ICD — Masks WHERE the model looks -> Forces learning from context",
             fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

### 5.3 Visualize AICD — Masking Background Noise

AICD is the opposite of ICD: it masks **unimportant regions** (background). This removes distracting noise and forces the model to focus on discriminative object features.

In [ ]:
aicd = AICD(
    model=model, target_layers=target_layers,
    threshold_percentile=60.0, explainer="opticam",
    use_cuda=False, cache=xai_cache,
    tile_size=8, fill_strategy="gaussian_blur",
    probability=1.0,
)

fig, axes = plt.subplots(2, 4, figsize=(14, 7))

for col, idx in enumerate(SAMPLE_INDICES[:4]):
    img_tensor, label = dataset[idx]
    img_np = (img_tensor.permute(1, 2, 0).numpy() * 255).astype(np.uint8)

    imgs_batch = np.expand_dims(img_np, 0)
    labels_batch = np.array([label])
    indices_batch = np.array([idx])
    aicd_result = aicd.apply_batch_with_labels(imgs_batch.copy(), labels_batch, indices_batch)

    axes[0, col].imshow(img_np)
    axes[0, col].set_title(STL10_CLASSES[label], fontsize=10)
    axes[0, col].axis("off")

    axes[1, col].imshow(aicd_result[0])
    axes[1, col].set_title("After AICD", fontsize=10)
    axes[1, col].axis("off")

axes[0, 0].set_ylabel("Original", fontsize=11, fontweight="bold")
axes[1, 0].set_ylabel("AICD Result\n(background masked)", fontsize=11, fontweight="bold")

fig.suptitle("AICD — Masks background noise -> Sharpens focus on objects",
             fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

### 5.4 ICD vs AICD — When to Use Which?

| Scenario | Use | Why |
|:---|:---|:---|
| Model overfits to a single feature (e.g. always looks at beak to identify birds) | **ICD** | Forces model to learn multiple features per class |
| Noisy backgrounds confuse the model (e.g. misclassifies based on sky colour) | **AICD** | Removes background noise, sharpens object features |
| Small dataset (<5K images) | **Both** | Maximum regularization effect |
| Large dataset with varied backgrounds | **ICD** | Background is already diverse, focus on feature diversity |
| Medical/industrial imaging with uniform backgrounds | **AICD** | Background carries little information |

**Pro tip:** BNNR tests **both** ICD and AICD as augmentation candidates and keeps whichever improves your metric more. You don't have to choose — just include both, and BNNR decides.

### 5.5 Fill Strategies

When ICD/AICD masks a tile, it fills it with one of these strategies:

In [ ]:
strategies = ["gaussian_blur", "local_mean", "noise", "solid"]
img_np = samples[0][0].copy()
label = samples[0][1]

fig, axes = plt.subplots(1, len(strategies) + 1, figsize=(3 * (len(strategies) + 1), 3))
axes[0].imshow(img_np)
axes[0].set_title("Original", fontsize=10)
axes[0].axis("off")

for i, strategy in enumerate(strategies):
    icd_s = ICD(
        model=model, target_layers=target_layers,
        threshold_percentile=50.0, explainer="opticam",
        use_cuda=False, cache=xai_cache,
        tile_size=8, fill_strategy=strategy,
        probability=1.0,
    )
    batch = np.expand_dims(img_np, 0)
    result = icd_s.apply_batch_with_labels(batch.copy(), np.array([label]), np.array([0]))
    axes[i + 1].imshow(result[0])
    axes[i + 1].set_title(strategy.replace("_", " ").title(), fontsize=10)
    axes[i + 1].axis("off")

fig.suptitle("ICD Fill Strategies — How masked tiles are replaced",
             fontsize=13, fontweight="bold")
plt.tight_layout()
plt.show()

print("'gaussian_blur' (default) preserves low-frequency structure.")
print("This is crucial — black rectangles create out-of-distribution artifacts.")
print("Blurred fill keeps the image realistic while masking fine details.")

---

## 6. Mixing with External Libraries

BNNR integrates seamlessly with torchvision, Albumentations, and Kornia. You can combine them as candidates — BNNR tests each one and selects the best.

In [ ]:
tv_jitter = TorchvisionAugmentation(
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.3),
    name_override="tv_color_jitter",
    probability=1.0,
)
show_augmentation(samples[2][0], tv_jitter, "Torchvision ColorJitter (via BNNR wrapper)", n_variants=3)

from bnnr import albumentations_available, AlbumentationsAugmentation
if albumentations_available():
    import albumentations as A
    albu_aug = AlbumentationsAugmentation(
        A.Compose([A.HorizontalFlip(p=0.5), A.RandomBrightnessContrast(p=0.8)]),
        name_override="albu_combo",
        probability=1.0,
    )
    show_augmentation(samples[2][0], albu_aug, "Albumentations Combo (via BNNR wrapper)", n_variants=3)
else:
    print("Albumentations not installed. Install with: pip install bnnr[albumentations]")

print("\nYou can combine ALL of these as candidates. BNNR tests each and keeps the winners!")

---

## 7. Automatic Selection — How BNNR Chooses

The key value of BNNR is that you **don't manually pick augmentations**. You provide candidates, and BNNR:

1. Trains a **baseline** (no augmentation)
2. Tests each candidate individually — trains for `m_epochs` and measures your metric
3. **Selects the best** (highest accuracy / lowest loss / highest mAP)
4. Stacks it on top and repeats with remaining candidates

This means:
- **Harmful augmentations are automatically rejected** (BNNR only keeps what improves your metric)
- **The order matters** — BNNR finds the best stacking sequence
- **You can be aggressive** with candidates — throw in 20+ augmentations, BNNR handles the rest

In [ ]:
auto_augs = auto_select_augmentations()

print(f"auto_select_augmentations() selected {len(auto_augs)} candidates:")
print()
for i, aug in enumerate(auto_augs, 1):
    gpu_tag = " [GPU]" if getattr(aug, 'device_compatible', False) else " [CPU]"
    print(f"  {i:2d}. {aug.name:<24s}  p={aug.probability:.2f}{gpu_tag}")

print(f"\nFeed these to BNNRTrainer and it will test each one.")
print(f"Only augmentations that IMPROVE your metric will be selected.")

---

## 8. Registry — All Available Augmentations

In [ ]:
all_names = AugmentationRegistry.list_all()
print(f"Total registered augmentations: {len(all_names)}\n")
for name in all_names:
    cls = AugmentationRegistry.get(name)
    gpu = "GPU" if getattr(cls, 'device_compatible', False) else "CPU"
    doc = (cls.__doc__ or "").strip().split("\n")[0][:60] if cls.__doc__ else ""
    print(f"  {name:<24s}  [{gpu}]  {doc}")

---

## Summary

| Category | Augmentations | What's Special |
|:---|:---|:---|
| **8 Built-in** | ChurchNoise, DifPresets, Drust, LuxferGlass, ProCAM, Smugs, TeaStains (7 unique) + BasicAug (standard transforms) | 7 unique to BNNR — realistic noise/degradation patterns |
| **XAI-Driven** | **ICD**, **AICD** | Uses saliency maps to mask important/unimportant regions — no other library has this |
| **Wrappers** | TorchvisionAugmentation, AlbumentationsAugmentation, KorniaAugmentation | Use your favourite library inside BNNR's selection framework |
| **Auto-Selection** | `auto_select_augmentations()` | BNNR picks the best candidates for your hardware |
| **Auto-Decision** | `BNNRTrainer.run()` | BNNR tests each candidate and keeps only those that improve your metric |

### Next Steps

- **Classification:** [Classification Demo Notebook](classification/bnnr_classification_demo.ipynb) — Full training with all augmentations + ICD/AICD
- **Detection:** [Detection Demo Notebook](detection/bnnr_detection_demo.ipynb) — Detection-specific augmentations + XAI
- **Custom Data:** [Custom Data Notebook](bnnr_custom_data.ipynb) — Bring your own images